In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

# Part 1

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
#drop cells with Borough not assigned
df_2 = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
# group multiple neighborhoods having same postal code
df_3 = df_2.groupby(['PostalCode', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
df_3['Neighborhood'] = df_3['Neighborhood'].str.replace('/', ',')
df_3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# rows of dataframe
df_3.shape

(103, 3)

# Part 2

In [8]:
!pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 10.3 MB/s eta 0:00:01


In [18]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.5 MB/s  eta 0:00:01


In [9]:
import pandas as pd
import numpy as np
import geocoder
print("Imported!")

Imported!


In [10]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.709020000000066, -79.36348999999996]

In [11]:
# Retrieving Postal Code Co-ordinates
postal_codes = df_3['PostalCode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [12]:
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_3['Latitude'] = df_coords['Latitude']
df_3['Longitude'] = df_coords['Longitude']

In [13]:
df_3[df_3.PostalCode == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493


In [14]:
df_3.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892
5,M1J,Scarborough,Scarborough Village,43.74446,-79.23117
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.72582,-79.26461
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.71289,-79.28506
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.72360,-79.23496
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.69510,-79.26466


# Part 3

In [15]:
print('The dataframe has {} boroughs'.format(
        len(df_3['Borough'].unique())
    )
)

The dataframe has 15 boroughs


In [20]:
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

address = 'Toronto , Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_3['Latitude'], df_3['Longitude'], df_3['Borough'], df_3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [22]:
scarborough_data = df_3[df_3['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [23]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


In [24]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Borough'], scarborough_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [25]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: RCJUSB0OBO1HFHKARQQED3BB3CFQW1H3LVBHPAZCRWX54M1K
CLIENT_SECRET:4IZ0EFPUEB24WKHF2MXNE3W0W13BYW4SVEQY4RGSYHCDDTJN


In [26]:
scarborough_data.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [27]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.811390000000074, -79.19661999999994.


In [28]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607792b98353ca1e4b440f6e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.815890004500076,
    'lng': -79.19039569057789},
   'sw': {'lat': 43.80688999550007, 'lng': -79.20284430942199}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label':

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [34]:
print(scarborough_venues.shape)
scarborough_venues.head()

(99, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.81139,-79.19662,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.76575,-79.17470,Canadian Carpet Cleaning and Janitorial Service,43.767731,-79.174240,Dry Cleaner
4,"Guildwood, Morningside, West Hill",43.76575,-79.17470,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


In [35]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,14,14,14,14,14,14
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,2,2,2,2,2,2
"Clarks Corners, Tam O'Shanter, Sullivan",17,17,17,17,17,17
"Cliffside, Cliffcrest, Scarborough Village West",9,9,9,9,9,9
"Dorset Park, Wexford Heights, Scarborough Town Centre",4,4,4,4,4,4
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",4,4,4,4,4,4
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [36]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


In [37]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Bubble Tea Shop,Bus Line,Bus Stop,Business Service,...,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Sushi Restaurant,Trail,Video Game Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
scarborough_onehot.shape

(99, 56)

In [39]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Bubble Tea Shop,Bus Line,Bus Stop,Business Service,...,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Sushi Restaurant,Trail,Video Game Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.071429,0.071429,0.000000,0.0,0.071429,0.000000,0.000000,0.0,...,0.000000,0.071429,0.071429,0.000000,0.0,0.071429,0.071429,0.0,0.000000,0.071429
1,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
2,Cedarbrae,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000000,0.058824,0.058824,0.0,0.000000,0.000000,0.058824,0.0,...,0.058824,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.058824,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000000,0.222222,0.000000,0.0,0.000000,0.111111,0.000000,0.0,...,0.000000,0.000000,0.000000,0.111111,0.0,0.000000,0.000000,0.0,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
9,"Malvern, Rouge",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [40]:
scarborough_grouped.shape

(16, 56)

In [41]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0     Chinese Restaurant  0.14
1  Vietnamese Restaurant  0.07
2           Skating Rink  0.07
3        Badminton Court  0.07
4          Grocery Store  0.07


----Birch Cliff, Cliffside West----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25
3        College Stadium  0.25
4            Auto Garage  0.00


----Cedarbrae----
                  venue  freq
0                 Trail   0.5
1           Gaming Cafe   0.5
2           Auto Garage   0.0
3                  Park   0.0
4  Hong Kong Restaurant   0.0


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0  Fast Food Restaurant  0.12
1              Pharmacy  0.12
2           Pizza Place  0.12
3            Hobby Shop  0.06
4        Discount Store  0.06


----Cliffside, Cliffcrest, Scarborough Village West----
            venue  freq
0  Ice Cream Shop  0.22
1     Pizza Place  0.11
2    Liquor 

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
#create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Vietnamese Restaurant,Shopping Mall,Hong Kong Restaurant,Discount Store,Department Store,Bubble Tea Shop,Grocery Store,Skating Rink,Badminton Court
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Gift Shop,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner
2,Cedarbrae,Trail,Gaming Cafe,Vietnamese Restaurant,College Stadium,Gift Shop,General Entertainment,Fast Food Restaurant,Electronics Store,Dry Cleaner,Discount Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pharmacy,Pizza Place,Fast Food Restaurant,Grocery Store,Bus Stop,Coffee Shop,Discount Store,Cantonese Restaurant,Sandwich Place,Hobby Shop
4,"Cliffside, Cliffcrest, Scarborough Village West",Ice Cream Shop,Liquor Store,Pharmacy,Discount Store,Pizza Place,Coffee Shop,Sandwich Place,Restaurant,Convenience Store,College Stadium


In [44]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 1], dtype=int32)

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,1.0,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Gift Shop,General Entertainment,Gaming Cafe,Electronics Store,Dry Cleaner,Discount Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875,4.0,Bar,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Gift Shop,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470,0.0,Park,Gym / Fitness Center,Dry Cleaner,Construction & Landscaping,Gift Shop,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Discount Store
3,M1G,Scarborough,Woburn,43.76812,-79.21761,0.0,Park,Mexican Restaurant,Business Service,Korean BBQ Restaurant,Coffee Shop,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892,0.0,Trail,Gaming Cafe,Vietnamese Restaurant,College Stadium,Gift Shop,General Entertainment,Fast Food Restaurant,Electronics Store,Dry Cleaner,Discount Store


In [46]:
scarborough_merged['Cluster Labels'] = scarborough_merged['Cluster Labels'].fillna(0)
scarborough_merged['Cluster Labels'] = scarborough_merged['Cluster Labels'].astype(int)

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# examine clusters

In [48]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0,Park,Gym / Fitness Center,Dry Cleaner,Construction & Landscaping,Gift Shop,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Discount Store
3,Scarborough,0,Park,Mexican Restaurant,Business Service,Korean BBQ Restaurant,Coffee Shop,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner
4,Scarborough,0,Trail,Gaming Cafe,Vietnamese Restaurant,College Stadium,Gift Shop,General Entertainment,Fast Food Restaurant,Electronics Store,Dry Cleaner,Discount Store
5,Scarborough,0,Grocery Store,Spa,Indian Restaurant,Restaurant,Park,College Stadium,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store
6,Scarborough,0,Coffee Shop,Discount Store,Department Store,Playground,College Stadium,Gift Shop,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store
7,Scarborough,0,Coffee Shop,Bakery,Intersection,Soccer Field,Bus Line,Metro Station,Construction & Landscaping,General Entertainment,Gaming Cafe,Fast Food Restaurant
8,Scarborough,0,Ice Cream Shop,Liquor Store,Pharmacy,Discount Store,Pizza Place,Coffee Shop,Sandwich Place,Restaurant,Convenience Store,College Stadium
9,Scarborough,0,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Gift Shop,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner
11,Scarborough,0,Auto Garage,Home Service,Convenience Store,Gym / Fitness Center,Gift Shop,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner
12,Scarborough,0,Chinese Restaurant,Vietnamese Restaurant,Shopping Mall,Hong Kong Restaurant,Discount Store,Department Store,Bubble Tea Shop,Grocery Store,Skating Rink,Badminton Court


In [49]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Gift Shop,General Entertainment,Gaming Cafe,Electronics Store,Dry Cleaner,Discount Store,Department Store


In [50]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,2,Pharmacy,Intersection,Vietnamese Restaurant,College Stadium,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner,Discount Store


In [51]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,3,Bakery,Gift Shop,Light Rail Station,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner


In [52]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4,Bar,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Gift Shop,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Dry Cleaner


In [56]:
scarborough_merged.shape

(17, 16)